In [5]:
from vgg16 import vgg16
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

In [4]:
class PruneConfig():
    def __init__(self):
        self.batch_size = 50
        self.epochs = 100
        self.lr = 0.01
        self.cuda = True
        self.seed = 42
        self.log_rate = 10
        self.log_file = "log.txt"
        self.sensitivity = 2
cfg = PruneConfig()

In [6]:
torch.manual_seed(cfg.seed)

In [7]:
if cfg.cuda:
    print("Using CUDA")
    torch.cuda.manual_seed(cfg.seed)
else:
    print("No CUDA")
kwargs = {'num_workers': 5, 'pin_memory': True} if cfg.cuda else {}

Using CUDA


### Use Toy MNIST Data

In [8]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True,
                                                         transform=transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,),(0.3081,))
                                                         ])),
                                          batch_size=cfg.batch_size,
                                          shuffle=True,
                                          **kwargs)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, download=True,
                                                         transform=transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,),(0.3081,))
                                                         ])),
                                          batch_size=cfg.batch_size,
                                          shuffle=False,
                                          **kwargs)

Processing...
Done!


In [11]:
dev = torch.device("cuda" if cfg.cuda else 'cpu')
model = vgg16(mask=True).to(dev)